# permits-data

Data analysis pipeline for construction permits in Los Angeles.

For more information:
https://data.lacity.org/A-Prosperous-City/Building-and-Safety-Permit-Information/yv23-pmwf

In [2]:
import os
import sys
import pandas as pd
import psycopg2

In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [4]:
sys.path[0] = '../'

## Import Data

In [5]:
raw_data = 'permits_raw.csv'

URL = "https://data.lacity.org/api/views/yv23-pmwf/rows.csv?accessType=DOWNLOAD"
DATA_PATH = sys.path[0] + 'data/raw/' + raw_data

In [10]:
#data = pd.read_csv(DATA_PATH)

/Users/gregory/anaconda3/envs/permits-data-env/lib/python3.5/site-packages/IPython/core/interactiveshell.py:3018: DtypeWarning: Columns (0,1,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [55]:
conn = psycopg2.connect(dbname='permits',
                       user='postgres',
                       password='password',
                        host='localhost', 
                        port=5432)

sql = 'SELECT * FROM permits_raw WHERE "Zip Code"=90027 LIMIT 10;'

In [56]:
data = pd.read_sql_query(sql, conn)

In [57]:
data.head()

,Assessor Book,Assessor Page,Assessor Parcel,Tract,Block,Lot,Reference # (Old Permit #),PCIS Permit #,Status,Status Date,Permit Type,Permit Sub-Type,Permit Category,Project Number,Event Code,Initiating Office,Issue Date,Address Start,Address Fraction Start,Address End,Address Fraction End,Street Direction,Street Name,Street Suffix,Suffix Direction,Unit Range Start,Unit Range End,Zip Code,Work Description,Valuation,Floor Area-L.A. Zoning Code Definition,# of Residential Dwelling Units,# of Accessory Dwelling Units,# of Stories,Contractor's Business Name,Contractor Address,Contractor City,Contractor State,License Type,License #,Principal First Name,Principal Middle Name,Principal Last Name,License Expiration Date,Applicant First Name,Applicant Last Name,Applicant Business Name,Applicant Address 1,Applicant Address 2,Applicant Address 3,Zone,Occupancy,Floor Area-L.A. Building Code Definition,Census Tract,Council District,Latitude/Longitude,Applicant Relationship,Existing Code,Proposed Code
0,5590,010,013,HOEGEE'S EAST HOLLYWOOD TRACT,None,36,None,17041-10000-42973,Permit Finaled,02/02/2018,Electrical,Commercial,No Plan Check,None,None,METRO,11/29/2017,1972,None,1972,None,N,HILLHURST,AVE,None,None,None,90027,None,None,None,None,None,None,KORMAN CONSTRUCTION INC,5042 WILSHIRE BLVD #388,LOS ANGELES,CA,B,1006832,BENJAMIN,SANGKU,LEE,08/31/2019,STEVE,OH,None,None,None,None,C4-1D,None,0,1891.01,4,"(34.10734, -118.28727)",Contractor,None,None
1,5591,012,017,TR 5832,None,79,None,18044-90000-04662,Permit Finaled,04/23/2018,HVAC,1 or 2 Family Dwelling,No Plan Check,None,None,INTERNET,04/19/2018,4115,None,4115,None,W,HOLLY KNOLL,DR,None,None,None,90027,None,None,None,None,None,None,BUILDING DOCTORS,5042 WILSHIRE BOULEVARD #268,LOS ANGELES,CA,C20,932241,DANIEL,CRAIG,THOMSEN,05/31/2018,DANIEL,THOMSEN,None,5042 WILSHIRE BLVD,SUITE 268,"LOS ANGELES, CA",R1-1,None,0,1952.02,4,"(34.10775, -118.2805)",Net Applicant,None,None
2,5434,019,025,IVANHOE,6,21,14VN86126,14047-20000-01600,CofO Issued,03/22/2016,Swimming-Pool/Spa,1 or 2 Family Dwelling,Plan Check,None,None,VAN NUYS,11/04/2014,3061,None,3061,None,N,ST GEORGE,ST,None,None,None,90027,new pool and spa per std. plan #270 (30' x 15'),25000.00,None,None,None,None,LUNA CUSTOM POOLS INC,2603 COLD CREEK AVENUE,ROSAMOND,CA,C53,979185,DENNIS,JAMES,SATTERFIELD,12/31/2014,RICARDO,FONSECA,None,4637 1/2 FIRST ST.,None,LOS ANGELES CA,R1-1,Misc. Occ. Group,0,1882.01,4,"(34.11233, -118.27299)",Agent for Owner,None,20
3,5591,021,014,WEST PORTION OF THE LICK TRACT = LOS FELIS RO,None,LT 36,13LA21167,13016-70000-17226,Permit Finaled,09/16/2013,Bldg-Alter/Repair,Commercial,Plan Check,None,None,SOUTH LA,08/29/2013,4151,None,4151,None,W,PROSPECT,AVE,None,Suite-130,None,90027,Tenant Improvement of 500 sq.ft on the 1st flo...,12000.00,None,None,None,None,P R CONSTRUCTION INC,1995 BATAVIA STREET,ORANGE,CA,B,673267,SEAN,FRANCIS,BRENNAN,03/31/2015,JIMMIE,DYER,None,515 S. FLOWER ST.,None,"LOS ANGELES, CA",[Q]M1-1D,None,None,1952.02,4,"(34.10191, -118.28217)",Agent for Owner,13,None
4,5430,008,008,TR 4587,None,9,18LA03130,18014-10000-01396,Issued,05/16/2018,Bldg-Addition,1 or 2 Family Dwelling,Plan Check,None,None,METRO,05/16/2018,2132,None,2132,None,N,MAYVIEW,DR,None,None,None,90027,1-STORY ADDITION (49 SF) TO (E) 2-STORY S.F.D....,25000.00,1629,0,None,0,OWNER-BUILDER,None,None,None,NA,0,None,None,None,01/01/2000,PAUL,VILLALOBOS,None,8252 NOREN ST.,None,"DOWNEY, CA",R1-1,None,49,1952.02,4,"(34.10057, -118.27636)",Agent for Owner,1,None


## Clean Data

In [58]:
def format_names(dataframe):
    # Replace whitespace with underscore
    dataframe.columns = dataframe.columns.str.replace(' ', '_')

    # Replace hyphen with underscore
    dataframe.columns = dataframe.columns.str.replace('-', '_')

    # Replace hashtag with No (short for number)
    dataframe.columns = dataframe.columns.str.replace('#', 'No')

    # Replace forward slash with underscore
    dataframe.columns = dataframe.columns.str.replace('/', '_')

    # Remove period
    dataframe.columns = dataframe.columns.str.replace('.', '')

    # Remove open parenthesis
    dataframe.columns = dataframe.columns.str.replace('(', '')

    # Remove closed parenthesis
    dataframe.columns = dataframe.columns.str.replace(')', '')

    # Remove apostrophe
    dataframe.columns = dataframe.columns.str.replace("'", '')
    
    return dataframe

In [61]:
data = format_names(data)

In [62]:
data

,Assessor_Book,Assessor_Page,Assessor_Parcel,Tract,Block,Lot,Reference_No_Old_Permit_No,PCIS_Permit_No,Status,Status_Date,Permit_Type,Permit_Sub_Type,Permit_Category,Project_Number,Event_Code,Initiating_Office,Issue_Date,Address_Start,Address_Fraction_Start,Address_End,Address_Fraction_End,Street_Direction,Street_Name,Street_Suffix,Suffix_Direction,Unit_Range_Start,Unit_Range_End,Zip_Code,Work_Description,Valuation,Floor_Area_LA_Zoning_Code_Definition,No_of_Residential_Dwelling_Units,No_of_Accessory_Dwelling_Units,No_of_Stories,Contractors_Business_Name,Contractor_Address,Contractor_City,Contractor_State,License_Type,License_No,Principal_First_Name,Principal_Middle_Name,Principal_Last_Name,License_Expiration_Date,Applicant_First_Name,Applicant_Last_Name,Applicant_Business_Name,Applicant_Address_1,Applicant_Address_2,Applicant_Address_3,Zone,Occupancy,Floor_Area_LA_Building_Code_Definition,Census_Tract,Council_District,Latitude_Longitude,Applicant_Relationship,Existing_Code,Proposed_Code
0,5590,010,013,HOEGEE'S EAST HOLLYWOOD TRACT,None,36,None,17041-10000-42973,Permit Finaled,02/02/2018,Electrical,Commercial,No Plan Check,None,None,METRO,11/29/2017,1972,None,1972,None,N,HILLHURST,AVE,None,None,None,90027,None,None,None,None,None,None,KORMAN CONSTRUCTION INC,5042 WILSHIRE BLVD #388,LOS ANGELES,CA,B,1006832,BENJAMIN,SANGKU,LEE,08/31/2019,STEVE,OH,None,None,None,None,C4-1D,None,0,1891.01,4,"(34.10734, -118.28727)",Contractor,None,None
1,5591,012,017,TR 5832,None,79,None,18044-90000-04662,Permit Finaled,04/23/2018,HVAC,1 or 2 Family Dwelling,No Plan Check,None,None,INTERNET,04/19/2018,4115,None,4115,None,W,HOLLY KNOLL,DR,None,None,None,90027,None,None,None,None,None,None,BUILDING DOCTORS,5042 WILSHIRE BOULEVARD #268,LOS ANGELES,CA,C20,932241,DANIEL,CRAIG,THOMSEN,05/31/2018,DANIEL,THOMSEN,None,5042 WILSHIRE BLVD,SUITE 268,"LOS ANGELES, CA",R1-1,None,0,1952.02,4,"(34.10775, -118.2805)",Net Applicant,None,None
2,5434,019,025,IVANHOE,6,21,14VN86126,14047-20000-01600,CofO Issued,03/22/2016,Swimming-Pool/Spa,1 or 2 Family Dwelling,Plan Check,None,None,VAN NUYS,11/04/2014,3061,None,3061,None,N,ST GEORGE,ST,None,None,None,90027,new pool and spa per std. plan #270 (30' x 15'),25000.00,None,None,None,None,LUNA CUSTOM POOLS INC,2603 COLD CREEK AVENUE,ROSAMOND,CA,C53,979185,DENNIS,JAMES,SATTERFIELD,12/31/2014,RICARDO,FONSECA,None,4637 1/2 FIRST ST.,None,LOS ANGELES CA,R1-1,Misc. Occ. Group,0,1882.01,4,"(34.11233, -118.27299)",Agent for Owner,None,20
3,5591,021,014,WEST PORTION OF THE LICK TRACT = LOS FELIS RO,None,LT 36,13LA21167,13016-70000-17226,Permit Finaled,09/16/2013,Bldg-Alter/Repair,Commercial,Plan Check,None,None,SOUTH LA,08/29/2013,4151,None,4151,None,W,PROSPECT,AVE,None,Suite-130,None,90027,Tenant Improvement of 500 sq.ft on the 1st flo...,12000.00,None,None,None,None,P R CONSTRUCTION INC,1995 BATAVIA STREET,ORANGE,CA,B,673267,SEAN,FRANCIS,BRENNAN,03/31/2015,JIMMIE,DYER,None,515 S. FLOWER ST.,None,"LOS ANGELES, CA",[Q]M1-1D,None,None,1952.02,4,"(34.10191, -118.28217)",Agent for Owner,13,None
4,5430,008,008,TR 4587,None,9,18LA03130,18014-10000-01396,Issued,05/16/2018,Bldg-Addition,1 or 2 Family Dwelling,Plan Check,None,None,METRO,05/16/2018,2132,None,2132,None,N,MAYVIEW,DR,None,None,None,90027,1-STORY ADDITION (49 SF) TO (E) 2-STORY S.F.D....,25000.00,1629,0,None,0,OWNER-BUILDER,None,None,None,NA,0,None,None,None,01/01/2000,PAUL,VILLALOBOS,None,8252 NOREN ST.,None,"DOWNEY, CA",R1-1,None,49,1952.02,4,"(34.10057, -118.27636)",Agent for Owner,1,None
5,5433,025,003,IVANHOE,25,18,None,16042-10001-23143,Permit Finaled,11/22/2016,Plumbing,Apartment,No Plan Check,None,None,METRO,11/02/2016,3745,None,3745,None,W,EVANS,ST,None,#2,None,90027,None,None,None,None,None,None,J B PLUMBING AND ROOTER,6114 FAIRFIELD STREET,LOS ANGELES,CA,C36,894504,JOSE,ARMANDO,BAIRES,04/30/2017,None,None,None,None,None,None,R2-1XL,None,0,1952.01,4,"(34.10665, -118.27385)",None,None,None
6,5588,022,012,TR 4441,None,30,None,18041-90000-00816,Permit Finaled,03/29/2018,

In [ ]:
data['Existing_Code'].value_counts()